In [1]:
%load_ext autoreload
%autoreload 3

# import gconcord as cc
import gconcorde as cce
import numpy as np

# import importlib
# importlib.reload(cc)

np.random.seed(3)

# true precision matrix
omega = np.zeros((3,3), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 3)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
data = np.random.multivariate_normal([0]*3, sigma, 500)

# sample covariance matrix
S = np.cov(data, rowvar=False)

In [2]:
def get_precision(X):
    return .5*(np.diag(np.diag(X)) @ X) + .5*(X.T @ np.diag(np.diag(X)))

In [3]:
omega_hat, hist = cce.pycceista(S, 0.5, epstol=1e-7, maxitr=1000)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.57936149  0.86200851  0.        ]
 [ 0.86200851  0.97038679 -0.861551  ]
 [ 0.         -0.861551    1.58339681]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.00000000e+00 1.33185340e-07 1.81275925e+00]
 [3.00000000e+00 2.10278388e-07 1.81275925e+00]
 [5.00000000e+00 7.07887237e-08 1.81275925e+00]]

iterations: 153
successive norm diff: 7.078872e-08
objective: 1.812759e+00


In [4]:
omega_hat2, hist2 = cce.cceista(S, 0.5, epstol=1e-7, maxitr=1000)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat2.todense()))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist2[-3:])))
print("iterations: %d" % (len(hist2)))
print("successive norm diff: %e" % (hist2[-1,1]))
print("objective: %e" % (hist2[-1,2]))


omega_hat:
 [[ 1.57936149  0.86200851  0.        ]
 [ 0.86200851  0.97038679 -0.861551  ]
 [ 0.         -0.861551    1.58339681]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.00000000e+00 1.33185340e-07 1.81275925e+00]
 [3.00000000e+00 2.10278388e-07 1.81275925e+00]
 [5.00000000e+00 7.07887237e-08 1.81275925e+00]]

iterations: 153
successive norm diff: 7.078872e-08
objective: 1.812759e+00


In [5]:
omega_hat3, hist3 = cce.cceista(S, 0.5, epstol=1e-7, maxitr=1000, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat3.todense()))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist3[-3:])))
print("iterations: %d" % (len(hist3)))
print("successive norm diff: %e" % (hist3[-1,1]))
print("objective: %e" % (hist3[-1,2]))

omega_hat:
 [[ 0.7462408   0.41210193  0.        ]
 [ 0.41210193  0.49396539 -0.41140869]
 [ 0.         -0.41140869  0.74753944]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.00000000e+00 1.55641448e-07 3.27145318e+00]
 [3.00000000e+00 2.47960967e-07 3.27145318e+00]
 [5.00000000e+00 8.70654888e-08 3.27145318e+00]]

iterations: 104
successive norm diff: 8.706549e-08
objective: 3.271453e+00


In [6]:
omega_hat, hist = cce.pycce_constant(S, 0.5, epstol=1e-7, maxitr=1000)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.57935733  0.86200657  0.        ]
 [ 0.86200657  0.97038538 -0.86154901]
 [ 0.         -0.86154901  1.58339259]]

hist: (inner_iter_count, delta_subg, objective)
 [[6.27000000e+02 1.03764933e-07 1.81275925e+00]
 [6.28000000e+02 1.01761717e-07 1.81275925e+00]
 [6.29000000e+02 9.97971846e-08 1.81275925e+00]]

iterations: 629
successive norm diff: 9.979718e-08
objective: 1.812759e+00


In [7]:
omega_hat, hist = cce.pycce_constant(S, 0.5, epstol=1e-7, maxitr=1000, penalize_diagonal=False)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))


omega_hat:
 [[ 1.57935733  0.86200657  0.        ]
 [ 0.86200657  0.97038538 -0.86154901]
 [ 0.         -0.86154901  1.58339259]]

hist: (inner_iter_count, delta_subg, objective)
 [[6.27000000e+02 1.03764933e-07 1.81275925e+00]
 [6.28000000e+02 1.01761717e-07 1.81275925e+00]
 [6.29000000e+02 9.97971846e-08 1.81275925e+00]]

iterations: 629
successive norm diff: 9.979718e-08
objective: 1.812759e+00


In [8]:
omega_hat5, hist5 = cce.cce_constant(S, 0.5, epstol=1e-7, maxitr=1000, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat5.todense()))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist5[-3:])))
print("iterations: %d" % (len(hist5)))
print("successive norm diff: %e" % (hist5[-1,1]))
print("objective: %e" % (hist5[-1,2]))


omega_hat:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

hist: (inner_iter_count, delta_subg, objective)
 [[0.         0.03010181        inf]
 [0.         0.02453336        inf]
 [0.         0.                inf]]

iterations: 35
successive norm diff: 0.000000e+00
objective: inf
